In [ ]:
import pydicom
from glob import glob
from random import randint
from copy import deepcopy
from datetime import datetime

import numpy as np
import pandas as pd

In [ ]:
pydicom.config.enforce_valid_values = True

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
multi_arc_plan = pydicom.read_file('MVISO_VMATNEWSPLIT.dcm', force=True)
beam_collimation_plan = pydicom.read_file('TinyFS_Rectangle.dcm', force=True)

In [ ]:
pydicom.Sequence

In [ ]:
beam_collimation = beam_collimation_plan.BeamSequence[0].ControlPointSequence[0].BeamLimitingDevicePositionSequence

In [ ]:
def display_control_points(dcm, beam_index):
    control_point_sequence = dcm.BeamSequence[beam_index].ControlPointSequence
    
    gantry_angles = [
        sequence.GantryAngle 
        for sequence in control_point_sequence
    ]

    gantry_rotation = [
        sequence.GantryRotationDirection 
        for sequence in control_point_sequence
    ]

    collimator_angles = [
        sequence.BeamLimitingDeviceAngle 
        for sequence in control_point_sequence
    ]

    collimator_rotation = [
        sequence.BeamLimitingDeviceRotationDirection 
        for sequence in control_point_sequence
    ]

    cumulative_meterset_weight = [
        sequence.CumulativeMetersetWeight 
        for sequence in control_point_sequence
    ]

    data = np.vstack([
        gantry_angles, gantry_rotation, 
        collimator_angles, collimator_rotation,
        cumulative_meterset_weight]).T
    return pd.DataFrame(data=data, columns=[
        'Gantry', 'Gantry Rotation', 
        'Collimator', 'Collimator Rotation', 'Meterset Weight'])

In [ ]:
def from_bipolar(angles: np.ndarray):
    ref = angles<0
    angles[ref] = angles[ref] + 360
    
    return angles

In [ ]:
gantry_step_size = 5.0

In [ ]:
gantry_beam_1 = from_bipolar(np.arange(-180, 181, gantry_step_size))
gantry_beam_1[0] = 180.1
gantry_beam_1[-1] = 179.9
gantry_beam_1

In [ ]:
coll_beam_1 = from_bipolar(np.arange(-180, 1, gantry_step_size/2))
coll_beam_1[0] = 180.1
coll_beam_1

In [ ]:
gantry_beam_2 = from_bipolar(np.arange(180, -181, -gantry_step_size))
gantry_beam_2[-1] = 180.1
gantry_beam_2[0] = 179.9
gantry_beam_2

In [ ]:
coll_beam_2 = from_bipolar(np.arange(180, -1, -gantry_step_size/2))
coll_beam_2[0] = 179.9
coll_beam_2

In [ ]:
num_cps = len(gantry_beam_2)
num_cps

meterset_weight = np.linspace(0, 1, num_cps)
meterset_weight

In [ ]:
total_mu = "300.000000"
dose_rate = "300"

In [ ]:
meterset_weight * float(total_mu)

In [ ]:
cpfirst_beam1 = multi_arc_plan.BeamSequence[0].ControlPointSequence[0]
cpmid_beam1 = multi_arc_plan.BeamSequence[0].ControlPointSequence[1]
cplast_beam1 = multi_arc_plan.BeamSequence[0].ControlPointSequence[-1]

cpfirst_beam1.BeamLimitingDevicePositionSequence = beam_collimation
cpmid_beam1.BeamLimitingDevicePositionSequence = beam_collimation
cplast_beam1.BeamLimitingDevicePositionSequence = beam_collimation

In [ ]:
cpfirst_beam1.BeamLimitingDeviceRotationDirection = 'CC'
cpmid_beam1.BeamLimitingDeviceRotationDirection = 'CC'

In [ ]:
cpfirst_beam1.DoseRateSet = dose_rate
cpmid_beam1.DoseRateSet = dose_rate
cplast_beam1.DoseRateSet = dose_rate

In [ ]:
cpfirst_beam2 = multi_arc_plan.BeamSequence[1].ControlPointSequence[0]
cpmid_beam2 = multi_arc_plan.BeamSequence[1].ControlPointSequence[1]
cplast_beam2 = multi_arc_plan.BeamSequence[1].ControlPointSequence[-1]

cpfirst_beam2.BeamLimitingDevicePositionSequence = beam_collimation
cpmid_beam2.BeamLimitingDevicePositionSequence = beam_collimation
cplast_beam2.BeamLimitingDevicePositionSequence = beam_collimation

In [ ]:
cpfirst_beam2.BeamLimitingDeviceRotationDirection = 'CW'
cpmid_beam2.BeamLimitingDeviceRotationDirection = 'CW'

In [ ]:
cpfirst_beam2.DoseRateSet = dose_rate
cpmid_beam2.DoseRateSet = dose_rate
cplast_beam2.DoseRateSet = dose_rate

In [ ]:
beam1 = pydicom.Sequence([])
beam2 = pydicom.Sequence([])


cpfirst_beam1.GantryAngle = str(gantry_beam_1[0])
cpfirst_beam2.GantryAngle = str(gantry_beam_2[0])

cpfirst_beam1.BeamLimitingDeviceAngle = str(coll_beam_1[0])
cpfirst_beam2.BeamLimitingDeviceAngle = str(coll_beam_2[0])

beam1.append(cpfirst_beam1)
beam2.append(cpfirst_beam2)

for i in range(1, num_cps - 1):
    cpmid_beam1.GantryAngle = str(gantry_beam_1[i])
    cpmid_beam2.GantryAngle = str(gantry_beam_2[i])

    cpmid_beam1.BeamLimitingDeviceAngle = str(coll_beam_1[i])
    cpmid_beam2.BeamLimitingDeviceAngle = str(coll_beam_2[i])
    
    cpmid_beam1.CumulativeMetersetWeight = str(round(meterset_weight[i], 6))
    cpmid_beam2.CumulativeMetersetWeight = str(round(meterset_weight[i], 6))
    
    cpmid_beam1.ControlPointIndex = str(i)
    cpmid_beam2.ControlPointIndex = str(i)
    
    beam1.append(deepcopy(cpmid_beam1))
    beam2.append(deepcopy(cpmid_beam2))
  

cplast_beam1.GantryAngle = str(gantry_beam_1[-1])
cplast_beam2.GantryAngle = str(gantry_beam_2[-1])

cplast_beam1.BeamLimitingDeviceAngle = str(coll_beam_1[-1])
cplast_beam2.BeamLimitingDeviceAngle = str(coll_beam_2[-1])

cplast_beam1.ControlPointIndex = str(num_cps-1)
cplast_beam2.ControlPointIndex = str(num_cps-1)

beam1.append(cplast_beam1)
beam2.append(cplast_beam2)

In [ ]:
new_plan = deepcopy(multi_arc_plan)

In [ ]:
new_plan.BeamSequence[0].ControlPointSequence = beam1
new_plan.BeamSequence[1].ControlPointSequence = beam2

In [ ]:
new_plan.BeamSequence[0].NumberOfControlPoints = str(num_cps)
new_plan.BeamSequence[1].NumberOfControlPoints = str(num_cps)

In [ ]:
new_plan.BeamSequence[0].BeamName = "WLutz-6MV-CW"
new_plan.BeamSequence[1].BeamName = "WLutz-6MV-CC"

In [ ]:
new_plan

In [ ]:
display_control_points(new_plan, 0)

In [ ]:
display_control_points(new_plan, 1)

In [ ]:
new_plan.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset = total_mu
new_plan.FractionGroupSequence[0].ReferencedBeamSequence[1].BeamMeterset = total_mu

In [ ]:
new_plan.RTPlanLabel = 'WLutzArc'
new_plan.RTPlanName = 'WLutzArc'

In [ ]:
# new_plan

In [ ]:
timestamp = datetime.now().isoformat(sep='_').split('.')[0].replace('-', '').replace(':', '')
plan_file_name = f"VMAT_MVISO_{timestamp}.dcm"
plan_file_name

In [ ]:
new_plan.save_as(plan_file_name)

In [ ]:
!echo {plan_file_name}

In [ ]:
!dcmsend 192.168.100.200 104 {plan_file_name} --read-dataset --aetitle CMS_SCU --call EOS_RTD -d

In [ ]:
# !dcmsend --help